# Main App

In [740]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [741]:
#|export
import pandas as pd
from flask import Flask, redirect, request, render_template, flash
from datamodel import Contact, Contacts, ContactErrors
import re

In [742]:

from IPython.display import display_html

In [743]:
#|export
app = Flask(__name__)
app.secret_key = "superdupersecret"


In [744]:
#|export
@app.get("/")
def index():
    return redirect("/contacts")

In [745]:
#|export
@app.get("/contacts")
def contacts():
    search = request.args.get("q")
    contact_set = None
    if search is not None: contact_set = Contacts().search(search)
    else: contact_set = Contacts().all()
    return render_template("index.html", contact_set=contact_set)

In [746]:
df = Contacts().db
df = df[df['id']==1].to_dict('records')[0]; df

{'firstname': 'Jane',
 'lastname': 'Smith',
 'phone': '555-5678',
 'email': 'jane.smith@example.com',
 'id': 1}

In [747]:
#|export
@app.get("/contacts/<int:id>")
def view(id:int):
    return render_template("view.html", contact=Contacts().get(id))

In [748]:
with app.app_context():
    print(view(2))

<!doctype html>
<html>
    <head>
        <title>Contact.App</title>
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@picocss/pico@2/css/pico.min.css">
        <script src="https://unpkg.com/htmx.org@2.0.1"></script>
    </head>
    <body hx-boost="true">
        <main class="container">
            <h1> CONTACTS.APP</h1>
            <h2> A demo contact application </h2>
            <hr>
            
    <article>
        <h1>Alice Johnson </h1>
        <div role="group">
            <div>555-8765</div>
            <div>alice.johnson@example.com</div>
        </div>
        <footer role="group">
            <a href="/contacts" role="button">Back</a>
            <a href="/contacts/2/edit" role="button">Edit</a>
        </footer>
    </article>
 
        </main>
    </body>
</html>


In [749]:
Contacts().get(2)

{'firstname': 'Alice',
 'lastname': 'Johnson',
 'phone': '555-8765',
 'email': 'alice.johnson@example.com',
 'id': 2}

In [750]:
#|export
@app.route("/contacts/<int:id>/edit", methods=['GET', 'POST'])
def edit(id):
    if request.method == 'GET':
        c_dict = Contacts().get(id)
        c = Contact()
        c.from_contacts_dict(c_dict)
        c.check_valid(duplicate_ok=True)
        return render_template('edit.html', contact=c)
    else:
        c_dict = Contacts().get(id)
        c = Contact()
        c.from_contacts_dict(c_dict)
        c.firstname = request.form['firstname']
        c.lastname = request.form['lastname']
        c.phone=request.form['phone']
        c.email=request.form['email']
        c.check_valid(duplicate_ok=True)
        if c.commit(duplicate_ok=True):
            flash("Updated Contract")
            return redirect("/contacts/"+str(id))
        else: return render_template('edit.html', contact=c)

In [751]:
#|export
@app.route("/contacts/<int:id>/delete", methods=['POST'])
def delete(id):
    Contacts().delete(id)
    flash("Contract Deleted")
    return redirect("/contacts")

In [752]:
#|export
@app.route("/contacts/<int:id>", methods=['DELETE'])
def delete_htmx(id):
    Contacts().delete(id)
    flash("Contract Deleted")
    return redirect("/contacts", 303)

In [753]:
contact=Contact(firstname=None, lastname=None, phone=None, email=None)
contact.firstname

In [754]:
#|export
@app.route("/contacts/new", methods=['GET'])
def contact_new_get():
    return render_template('new.html', contact=Contact(firstname=None, lastname=None, phone=None, email=None))

In [755]:
#|export
@app.route("/contacts/new", methods=['POST'])
def contact_new():
    c = Contact(firstname=request.form['firstname'], 
                lastname=request.form['lastname'],
                phone=request.form['phone'],
                email=request.form['email'])
    c.check_valid(duplicate_ok=False)
    if c.commit(duplicate_ok=False):
        flash("New contract created")
        return redirect("/contacts")
    else: 
        print(c)
        return render_template('new.html', contact=c)

In [762]:
#|eval:false
from nbdev.export import nb_export
nb_export("01_main.ipynb", lib_path=".", name='main')